In [25]:
#!Nutures
#!20200601
#!Trans data format

import pandas as pd
import multiprocessing as mp
import sys
import os

## data format
def trans_data_format(df_index, df_group, output_n):
    df_result = pd.DataFrame(columns=['class', 'mc',  'all_chrom', "all_chrom_number"])
    for index, row in df_index.iterrows():
        class_id, mc_id= row['class'], row['mc']
        numbers = df_group.get_group((class_id, mc_id)).shape[0]
        group_chrom = ";".join(set(df_group.get_group((class_id, mc_id))['chrom'].tolist()))
        df_result.loc[df_result.shape[0]] = [class_id, mc_id, group_chrom, numbers]
    # write the result
    df_result.to_csv(output_n, sep='\t', index=False)
## parallel
def run_parallel(cpu_number, df_groupid, df_group):
    gaps = int(df_groupid.shape[0]/cpu_number)
    end = 0
    datas = []
    print(df_groupid.shape[0])
    for n in range(cpu_number):
        start = end
        if n == cpu_number-1:
            end = df_groupid.shape[0]
        else:
            end += gaps
        df_index_cpu = df_groupid.iloc[start:end, ]
        print(start, end)
        output_name = "thread_{}.txt".format(n)
        p = mp.Process(target=trans_data_format, args=(df_index_cpu, df_group, output_name))
        datas.append(p)
        p.start()
    for p in datas:
        p.join()
## data concat
def concat(cpu_number, output_f):
    df_tmp = pd.DataFrame()
    for k in range(cpu_number):
        names = "thread_{0}.txt".format(k)
        df = pd.read_csv(names, sep='\t')
        df_tmp = df_tmp.append(df, ignore_index=True)
        os.remove(names)
    df_tmp.to_csv(output_f, sep='\t', index=False)

def main(argv):
    input_f = sys.argv[1]
    output_f = sys.argv[2]
    cpu_number = int(sys.argv[3])
    df = pd.read_csv(input_f, sep='\t', names=['class', 'mc', 'chrom', 'FDR'])

    ## get group id
    df_groupid = df[['class', 'mc']].drop_duplicates(keep='first').copy()
    ## data group
    df_group = df.groupby(by=['class', 'mc'])
    run_parallel(cpu_number, df_groupid, df_group)
    concat(cpu_number, output_f)
if __name__ == '__main__':
    main(sys.argv[1:])